# Analysis of LLM Labels

- Load LLM-labelled data
- Calculate the ratio of posts that are labelled prediction and non-predicton

In [1]:
import os
import sys
import json

import pandas as pd

from tqdm import tqdm

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# print(notebook_dir)
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

from data_processing import DataProcessing
from text_generation_models import TextGenerationModelFactory

In [2]:
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_rows', None)

## Load Bluesky Data

- Data with the model + label + majority vote label

In [3]:
path_to_data = "../data/open_measures/bluesky_all_llm_labels/bluesky-v10.json"
full_path = os.path.join(notebook_dir, path_to_data)
df = pd.read_json(full_path)
df

,sentence,model,label,meta_data,majority_vote
0,Wow! Nepal confirmed as one of 16 finalists for the #AFCU16Championship football slated for September 2016 in India.\n\nthe-afc.com/u-16-champions…,gemma2-9b-it,not a prediction,"{'data_range_per_file': 0, '$type': 'app.bsky.feed.post', 'author': 'did:plc:gc2uaayj6jhpp36xi7shstd7', 'authorProfile': '{'did': 'did:plc:gc2uaayj6jhpp36xi7shstd7', 'handle': 'ujjwal.com.np', 'followers': 226, 'indexed_at': '2024-11-22T03:37:11.716Z', 'following': 289, 'banner': 'https://cdn.bsky.app/img/banner/plain/did:plc:gc2uaayj6jhpp36xi7shstd7/bafkreiha5jbnooeza33jdu7o3aje7j3wl7ucvs65lsqkt2i72wofbmxfea@jpeg', 'description': ""Director https://Journalism.Academy, Nepal | Researcher, Trainer & Dreamer | Libertarian | Past: BBC, Annapurna Post, Republica, The Kathmandu Post, UNESCO & IFJ | Kritabi's Baba"", 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:gc2uaayj6jhpp36xi7shstd7/bafkreibjhs2xwb3cykvbgctisi3wnm2nrsvpkgyxmtjkorfktkxr23ecyq@jpeg', 'display_name': 'Ujjwal Achary...",not a prediction
1,Wow! Nepal confirmed as one of 16 finalists for the #AFCU16Championship football slated for September 2016 in India.\n\nthe-afc.com/u-16-champions…,llama-3.1-8b-instant,prediction,"{'data_range_per_file': 0, '$type': 'app.bsky.feed.post', 'author': 'did:plc:gc2uaayj6jhpp36xi7shstd7', 'authorProfile': '{'did': 'did:plc:gc2uaayj6jhpp36xi7shstd7', 'handle': 'ujjwal.com.np', 'followers': 226, 'indexed_at': '2024-11-22T03:37:11.716Z', 'following': 289, 'banner': 'https://cdn.bsky.app/img/banner/plain/did:plc:gc2uaayj6jhpp36xi7shstd7/bafkreiha5jbnooeza33jdu7o3aje7j3wl7ucvs65lsqkt2i72wofbmxfea@jpeg', 'description': ""Director https://Journalism.Academy, Nepal | Researcher, Trainer & Dreamer | Libertarian | Past: BBC, Annapurna Post, Republica, The Kathmandu Post, UNESCO & IFJ | Kritabi's Baba"", 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:gc2uaayj6jhpp36xi7shstd7/bafkreibjhs2xwb3cykvbgctisi3wnm2nrsvpkgyxmtjkorfktkxr23ecyq@jpeg', 'display_name': 'Ujjwal Achary...",not a prediction
2,Wow! Nepal confirmed as one of 16 finalists for the #AFCU16Championship football slated for September 2016 in India.\n\nthe-afc.com/u-16-champions…,llama-3.3-70b-versatile,not a prediction,"{'data_range_per_file': 0, '$type': 'app.bsky.feed.post', 'author': 'did:plc:gc2uaayj6jhpp36xi7shstd7', 'authorProfile': '{'did': 'did:plc:gc2uaayj6jhpp36xi7shstd7', 'handle': 'ujjwal.com.np', 'followers': 226, 'indexed_at': '2024-11-22T03:37:11.716Z', 'following': 289, 'banner': 'https://cdn.bsky.app/img/banner/plain/did:plc:gc2uaayj6jhpp36xi7shstd7/bafkreiha5jbnooeza33jdu7o3aje7j3wl7ucvs65lsqkt2i72wofbmxfea@jpeg', 'description': ""Director https://Journalism.Academy, Nepal | Researcher, Trainer & Dreamer | Libertarian | Past: BBC, Annapurna Post, Republica, The Kathmandu Post, UNESCO & IFJ | Kritabi's Baba"", 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:gc2uaayj6jhpp36xi7shstd7/bafkreibjhs2xwb3cykvbgctisi3wnm2nrsvpkgyxmtjkorfktkxr23ecyq@jpeg', 'display_name': 'Ujjwal Achary...",not a prediction
3,Wow! Nepal confirmed as one of 16 finalists for the #AFCU16Championship football slated for September 2016 in India.\n\nthe-afc.com/u-16-champions…,meta-llama/llama-guard-4-12b,safe,"{'data_range_per_file': 0, '$type': 'app.bsky.feed.post', 'author': 'did:plc:gc2uaayj6jhpp36xi7shstd7', 'authorProfile': '{'did': 'did:plc:gc2uaayj6jhpp36xi7shstd7', 'handle': 'ujjwal.com.np', 'followers': 226, 'indexed_at': '2024-11-22T03:37:11.716Z', 'following': 289, 'banner': 'https://cdn.bsky.app/img/banner/plain/did:plc:gc2uaayj6jhpp36xi7shstd7/bafkreiha5jbnooeza33jdu7o3aje7j3wl7ucvs65lsqkt2i72wofbmxfea@jpeg', 'description': ""Director https://Journalism.Academy, Nepal | Researcher, Trainer & Dreamer | Libertarian | Past: BBC, Annapurna Post, Republica, The Kathmandu Post, UNESCO & IFJ | Kritabi's Baba"", 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:gc2uaayj6jhpp36xi7shstd7/bafkreibjhs2xwb3cykvbgctisi3wnm2nrsvpkgyxmtjkorfktkx

In [11]:
len(df) / 4

200.75

## Filter for Non-Predictions and Predictions

In [13]:
filt_non_prediction = (df['majority_vote'] != 'prediction')
non_predictions_df = df[filt_non_prediction]
non_predictions_df.head(7)

,sentence,model,label,meta_data,majority_vote
0,Wow! Nepal confirmed as one of 16 finalists for the #AFCU16Championship football slated for September 2016 in India.\n\nthe-afc.com/u-16-champions…,gemma2-9b-it,not a prediction,"{'data_range_per_file': 0, '$type': 'app.bsky.feed.post', 'author': 'did:plc:gc2uaayj6jhpp36xi7shstd7', 'authorProfile': '{'did': 'did:plc:gc2uaayj6jhpp36xi7shstd7', 'handle': 'ujjwal.com.np', 'followers': 226, 'indexed_at': '2024-11-22T03:37:11.716Z', 'following': 289, 'banner': 'https://cdn.bsky.app/img/banner/plain/did:plc:gc2uaayj6jhpp36xi7shstd7/bafkreiha5jbnooeza33jdu7o3aje7j3wl7ucvs65lsqkt2i72wofbmxfea@jpeg', 'description': ""Director https://Journalism.Academy, Nepal | Researcher, Trainer & Dreamer | Libertarian | Past: BBC, Annapurna Post, Republica, The Kathmandu Post, UNESCO & IFJ | Kritabi's Baba"", 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:gc2uaayj6jhpp36xi7shstd7/bafkreibjhs2xwb3cykvbgctisi3wnm2nrsvpkgyxmtjkorfktkxr23ecyq@jpeg', 'display_name': 'Ujjwal Achary...",not a prediction
1,Wow! Nepal confirmed as one of 16 finalists for the #AFCU16Championship football slated for September 2016 in India.\n\nthe-afc.com/u-16-champions…,llama-3.1-8b-instant,prediction,"{'data_range_per_file': 0, '$type': 'app.bsky.feed.post', 'author': 'did:plc:gc2uaayj6jhpp36xi7shstd7', 'authorProfile': '{'did': 'did:plc:gc2uaayj6jhpp36xi7shstd7', 'handle': 'ujjwal.com.np', 'followers': 226, 'indexed_at': '2024-11-22T03:37:11.716Z', 'following': 289, 'banner': 'https://cdn.bsky.app/img/banner/plain/did:plc:gc2uaayj6jhpp36xi7shstd7/bafkreiha5jbnooeza33jdu7o3aje7j3wl7ucvs65lsqkt2i72wofbmxfea@jpeg', 'description': ""Director https://Journalism.Academy, Nepal | Researcher, Trainer & Dreamer | Libertarian | Past: BBC, Annapurna Post, Republica, The Kathmandu Post, UNESCO & IFJ | Kritabi's Baba"", 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:gc2uaayj6jhpp36xi7shstd7/bafkreibjhs2xwb3cykvbgctisi3wnm2nrsvpkgyxmtjkorfktkxr23ecyq@jpeg', 'display_name': 'Ujjwal Achary...",not a prediction
2,Wow! Nepal confirmed as one of 16 finalists for the #AFCU16Championship football slated for September 2016 in India.\n\nthe-afc.com/u-16-champions…,llama-3.3-70b-versatile,not a prediction,"{'data_range_per_file': 0, '$type': 'app.bsky.feed.post', 'author': 'did:plc:gc2uaayj6jhpp36xi7shstd7', 'authorProfile': '{'did': 'did:plc:gc2uaayj6jhpp36xi7shstd7', 'handle': 'ujjwal.com.np', 'followers': 226, 'indexed_at': '2024-11-22T03:37:11.716Z', 'following': 289, 'banner': 'https://cdn.bsky.app/img/banner/plain/did:plc:gc2uaayj6jhpp36xi7shstd7/bafkreiha5jbnooeza33jdu7o3aje7j3wl7ucvs65lsqkt2i72wofbmxfea@jpeg', 'description': ""Director https://Journalism.Academy, Nepal | Researcher, Trainer & Dreamer | Libertarian | Past: BBC, Annapurna Post, Republica, The Kathmandu Post, UNESCO & IFJ | Kritabi's Baba"", 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:gc2uaayj6jhpp36xi7shstd7/bafkreibjhs2xwb3cykvbgctisi3wnm2nrsvpkgyxmtjkorfktkxr23ecyq@jpeg', 'display_name': 'Ujjwal Achary...",not a prediction
3,Wow! Nepal confirmed as one of 16 finalists for the #AFCU16Championship football slated for September 2016 in India.\n\nthe-afc.com/u-16-champions…,meta-llama/llama-guard-4-12b,safe,"{'data_range_per_file': 0, '$type': 'app.bsky.feed.post', 'author': 'did:plc:gc2uaayj6jhpp36xi7shstd7', 'authorProfile': '{'did': 'did:plc:gc2uaayj6jhpp36xi7shstd7', 'handle': 'ujjwal.com.np', 'followers': 226, 'indexed_at': '2024-11-22T03:37:11.716Z', 'following': 289, 'banner': 'https://cdn.bsky.app/img/banner/plain/did:plc:gc2uaayj6jhpp36xi7shstd7/bafkreiha5jbnooeza33jdu7o3aje7j3wl7ucvs65lsqkt2i72wofbmxfea@jpeg', 'description': ""Director https://Journalism.Academy, Nepal | Researcher, Trainer & Dreamer | Libertarian | Past: BBC, Annapurna Post, Republica, The Kathmandu Post, UNESCO & IFJ | Kritabi's Baba"", 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:gc2uaayj6jhpp36xi7shstd7/bafkreibjhs2xwb3cykvbgctisi3wnm2nrsvpkgyxmtjkorfktkx

In [4]:
filt_prediction = (df['majority_vote'] == 'prediction')
predictions_df = df[filt_prediction]
predictions_df

,sentence,model,label,meta_data,majority_vote
8,"2016 Super Bowl odds: Patriots, Cards begin playoffs as co-favorites http://www.cbssports.com/nfl/eye-on-football/25442860/super-bowl-odds-patriots-cardinals-begin-playoffs-as-co-favorites via @CBSSports",gemma2-9b-it,not a prediction,"{'data_range_per_file': 2, '$type': 'app.bsky.feed.post', 'author': 'did:plc:ahok3cgd4c7rhnwrnofapqls', 'authorProfile': '{'_id': 'did:plc:ahok3cgd4c7rhnwrnofapqls', '_index': 'smat-bluesky-users', 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:ahok3cgd4c7rhnwrnofapqls/bafkreic5yiylpra6qg2hpluhkktuiggzesdtc25s6xhhpitqueiicso27e@jpeg', 'banner': 'https://cdn.bsky.app/img/banner/plain/did:plc:ahok3cgd4c7rhnwrnofapqls/bafkreia47gyvqcq2mcndvv2dtkwkepudxmwfk6ca4rujpwfynwn6onibsa@jpeg', 'description': 'Find my profile at https://vivensjoachim.brandyourself.com/\nFind my community works at: https://mibaleavantou.com, or\nhttp://vjmvp.com/index.php, and some other works at: https://ntkvideo.com/\nBorn in Haiti in 1970. Migrated to the USA in 1994. Live in CT', 'did': 'did:plc:ahok3cg...",prediction
9,"2016 Super Bowl odds: Patriots, Cards begin playoffs as co-favorites http://www.cbssports.com/nfl/eye-on-football/25442860/super-bowl-odds-patriots-cardinals-begin-playoffs-as-co-favorites via @CBSSports",llama-3.1-8b-instant,prediction,"{'data_range_per_file': 2, '$type': 'app.bsky.feed.post', 'author': 'did:plc:ahok3cgd4c7rhnwrnofapqls', 'authorProfile': '{'_id': 'did:plc:ahok3cgd4c7rhnwrnofapqls', '_index': 'smat-bluesky-users', 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:ahok3cgd4c7rhnwrnofapqls/bafkreic5yiylpra6qg2hpluhkktuiggzesdtc25s6xhhpitqueiicso27e@jpeg', 'banner': 'https://cdn.bsky.app/img/banner/plain/did:plc:ahok3cgd4c7rhnwrnofapqls/bafkreia47gyvqcq2mcndvv2dtkwkepudxmwfk6ca4rujpwfynwn6onibsa@jpeg', 'description': 'Find my profile at https://vivensjoachim.brandyourself.com/\nFind my community works at: https://mibaleavantou.com, or\nhttp://vjmvp.com/index.php, and some other works at: https://ntkvideo.com/\nBorn in Haiti in 1970. Migrated to the USA in 1994. Live in CT', 'did': 'did:plc:ahok3cg...",prediction
10,"2016 Super Bowl odds: Patriots, Cards begin playoffs as co-favorites http://www.cbssports.com/nfl/eye-on-football/25442860/super-bowl-odds-patriots-cardinals-begin-playoffs-as-co-favorites via @CBSSports",llama-3.3-70b-versatile,prediction,"{'data_range_per_file': 2, '$type': 'app.bsky.feed.post', 'author': 'did:plc:ahok3cgd4c7rhnwrnofapqls', 'authorProfile': '{'_id': 'did:plc:ahok3cgd4c7rhnwrnofapqls', '_index': 'smat-bluesky-users', 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:ahok3cgd4c7rhnwrnofapqls/bafkreic5yiylpra6qg2hpluhkktuiggzesdtc25s6xhhpitqueiicso27e@jpeg', 'banner': 'https://cdn.bsky.app/img/banner/plain/did:plc:ahok3cgd4c7rhnwrnofapqls/bafkreia47gyvqcq2mcndvv2dtkwkepudxmwfk6ca4rujpwfynwn6onibsa@jpeg', 'description': 'Find my profile at https://vivensjoachim.brandyourself.com/\nFind my community works at: https://mibaleavantou.com, or\nhttp://vjmvp.com/index.php, and some other works at: https://ntkvideo.com/\nBorn in Haiti in 1970. Migrated to the USA in 1994. Live in CT', 'did': 'did:plc:ahok3cg...",prediction
11,"2016 Super Bowl odds: Patriots, Cards begin playoffs as co-favorites http://www.cbssports.com/nfl/eye-on-football/25442860/super-bowl-odds-patriots-cardinals-begin-playoffs-as-co-favorites via @CBSSports",meta-llama/llama-guard-4-12b,safe,"{'data_range_per_file': 2, '$type': 'app.bsky.feed.post', 'author': 'did:plc:ahok3cgd4c7rhnwrnofapqls', 'authorProfile': '{'_id': 'did:plc:ahok3cgd4c7rhnwrnofapqls', '_index': 'smat-bluesky-users', 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:ahok3cgd4c7rhnwrnofapqls/bafkreic5yiylpra6qg2hpluhkktuiggzesdtc25s6xhhpitqueiicso27e@jpeg', 'banner': 'https://cdn.bsky.app/img/banner/plain/did:plc:ahok3cgd4c7rhnwrnofapqls/bafkreia47gyvqcq2mcndvv2dtkwkepudxmwfk6ca4rujpwfynwn6onibsa@jpeg', 'description': 'Find my profile at https://vivensjoac

## Extract metadata + Join with predictions

In [25]:
user_prediction_dfs = []
error_processing_users= []
for row_idx, row in predictions_df.iterrows():
    # print(row_idx)
    row_meta_data = row['meta_data']

    # print(row_meta_data.keys(), "\n")
    predictions_df.loc[row_idx, '$type'] = row_meta_data['$type']
    predictions_df.loc[row_idx, 'author'] = row_meta_data['author']
    predictions_df.loc[row_idx, 'date_time'] = row_meta_data['created_at']
    try: 
        json_to_str = row_meta_data['authorProfile'].replace("'", '"') # Convert single quotes to double quotes to make it valid JSON
        json_obj = json.loads(json_to_str) # Parse the JSON string
        user_series = pd.Series(json_obj)
        # print(user_series)
        try: 
            # print(row_idx, "---", user_series.keys())
            if 'display_name' in user_series.keys():
                predictions_df.loc[row_idx, 'display_name'] = user_series['display_name']
            if 'handle' in user_series.keys():
                predictions_df.loc[row_idx, 'handle'] = user_series['handle']
            if 'followers' in user_series.keys():    
                predictions_df.loc[row_idx, '#followers'] = user_series['followers']
            if 'following' in user_series.keys():
                predictions_df.loc[row_idx, '#following'] = user_series['following']
        except ValueError as e:
            user_to_error = (row_meta_data['display_name'], e)
            error_processing_users.append(user_to_error)
        continue

    except ValueError as e:
        user_to_error = (row_meta_data['author'], e)
        error_processing_users.append(user_to_error)
        continue
    except AttributeError as e:
        user_to_error = (row_meta_data['author'], e)
        error_processing_users.append(user_to_error)
        continue

In [26]:
predictions_df.head(32)

,sentence,model,label,meta_data,majority_vote,$type,author,date_time,display_name,handle,#followers,#following
8,"2016 Super Bowl odds: Patriots, Cards begin playoffs as co-favorites http://www.cbssports.com/nfl/eye-on-football/25442860/super-bowl-odds-patriots-cardinals-begin-playoffs-as-co-favorites via @CBSSports",gemma2-9b-it,not a prediction,"{'data_range_per_file': 2, '$type': 'app.bsky.feed.post', 'author': 'did:plc:ahok3cgd4c7rhnwrnofapqls', 'authorProfile': '{'_id': 'did:plc:ahok3cgd4c7rhnwrnofapqls', '_index': 'smat-bluesky-users', 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:ahok3cgd4c7rhnwrnofapqls/bafkreic5yiylpra6qg2hpluhkktuiggzesdtc25s6xhhpitqueiicso27e@jpeg', 'banner': 'https://cdn.bsky.app/img/banner/plain/did:plc:ahok3cgd4c7rhnwrnofapqls/bafkreia47gyvqcq2mcndvv2dtkwkepudxmwfk6ca4rujpwfynwn6onibsa@jpeg', 'description': 'Find my profile at https://vivensjoachim.brandyourself.com/\nFind my community works at: https://mibaleavantou.com, or\nhttp://vjmvp.com/index.php, and some other works at: https://ntkvideo.com/\nBorn in Haiti in 1970. Migrated to the USA in 1994. Live in CT', 'did': 'did:plc:ahok3cg...",prediction,app.bsky.feed.post,did:plc:ahok3cgd4c7rhnwrnofapqls,None,Vivens Joachim,vjoachim.bsky.social,59.0,215.0
9,"2016 Super Bowl odds: Patriots, Cards begin playoffs as co-favorites http://www.cbssports.com/nfl/eye-on-football/25442860/super-bowl-odds-patriots-cardinals-begin-playoffs-as-co-favorites via @CBSSports",llama-3.1-8b-instant,prediction,"{'data_range_per_file': 2, '$type': 'app.bsky.feed.post', 'author': 'did:plc:ahok3cgd4c7rhnwrnofapqls', 'authorProfile': '{'_id': 'did:plc:ahok3cgd4c7rhnwrnofapqls', '_index': 'smat-bluesky-users', 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:ahok3cgd4c7rhnwrnofapqls/bafkreic5yiylpra6qg2hpluhkktuiggzesdtc25s6xhhpitqueiicso27e@jpeg', 'banner': 'https://cdn.bsky.app/img/banner/plain/did:plc:ahok3cgd4c7rhnwrnofapqls/bafkreia47gyvqcq2mcndvv2dtkwkepudxmwfk6ca4rujpwfynwn6onibsa@jpeg', 'description': 'Find my profile at https://vivensjoachim.brandyourself.com/\nFind my community works at: https://mibaleavantou.com, or\nhttp://vjmvp.com/index.php, and some other works at: https://ntkvideo.com/\nBorn in Haiti in 1970. Migrated to the USA in 1994. Live in CT', 'did': 'did:plc:ahok3cg...",prediction,app.bsky.feed.post,did:plc:ahok3cgd4c7rhnwrnofapqls,None,Vivens Joachim,vjoachim.bsky.social,59.0,215.0
10,"2016 Super Bowl odds: Patriots, Cards begin playoffs as co-favorites http://www.cbssports.com/nfl/eye-on-football/25442860/super-bowl-odds-patriots-cardinals-begin-playoffs-as-co-favorites via @CBSSports",llama-3.3-70b-versatile,prediction,"{'data_range_per_file': 2, '$type': 'app.bsky.feed.post', 'author': 'did:plc:ahok3cgd4c7rhnwrnofapqls', 'authorProfile': '{'_id': 'did:plc:ahok3cgd4c7rhnwrnofapqls', '_index': 'smat-bluesky-users', 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:ahok3cgd4c7rhnwrnofapqls/bafkreic5yiylpra6qg2hpluhkktuiggzesdtc25s6xhhpitqueiicso27e@jpeg', 'banner': 'https://cdn.bsky.app/img/banner/plain/did:plc:ahok3cgd4c7rhnwrnofapqls/bafkreia47gyvqcq2mcndvv2dtkwkepudxmwfk6ca4rujpwfynwn6onibsa@jpeg', 'description': 'Find my profile at https://vivensjoachim.brandyourself.com/\nFind my community works at: https://mibaleavantou.com, or\nhttp://vjmvp.com/index.php, and some other works at: https://ntkvideo.com/\nBorn in Haiti in 1970. Migrated to the USA in 1994. Live in CT', 'did': 'did:plc:ahok3cg...",prediction,app.bsky.feed.post,did:plc:ahok3cgd4c7rhnwrnofapqls,None,Vivens Joachim,vjoachim.bsky.social,59.0,215.0
11,"2016 Super Bowl odds: Patriots, Cards begin playoffs as co-favorites http://www.cbssports.com/nfl/eye-on-football/25442860/super-bowl-odds-patriots-cardinals-begin-playoffs-as-co-favorites via @CBSSports",meta-llama/llama-guard-4-12b,safe,"{'data_range_per_file': 2, '$type': 'app.bsky.feed.post', 'author': 'did:plc:ahok3cgd4c7rhnwrnofapqls', 'authorProfile': '{'_id': 'did:plc:ahok3cgd4c7rhnwrnofapqls', '_inde

In [29]:
user_prediction = predictions_df.loc[:, ['display_name', 'sentence', 'majority_vote']]
bluesky_user_predictions = user_prediction.drop_duplicates(subset=["display_name", "sentence"])
bluesky_user_predictions

,display_name,sentence,majority_vote
8,Vivens Joachim,"2016 Super Bowl odds: Patriots, Cards begin playoffs as co-favorites http://www.cbssports.com/nfl/eye-on-football/25442860/super-bowl-odds-patriots-cardinals-begin-playoffs-as-co-favorites via @CBSSports",prediction
20,Anthony Rizzo,Is there a point to the rest of the #NFL season...or should we just get ready to cheer against the #Patriots in the Super Bowl?,prediction
24,NaN,I'll be shocked if Duke doesn't win the national championship this season.,prediction
36,NaN,(1969) The “League Championship Series” will water down the regular season,prediction
48,NaN,All SEC national championship game gonna expand the playoffs real quick,prediction
60,NaN,"Season ticket renewed... now, @stokecity, I live in Northamptonshire, it’s a two hour drive, I can’t afford the drive to all those extra championship games, nor can I stay up ‘til 1.00 am for all those weekday games so can we please stay in the premier league. Much obliged",prediction
64,NaN,I am resigned to championship football next season. I can’t see us winning any of the remaining games and whilst seven draws might be enough I can’t see that either. I just hope we play football next season rather than giving up at the first goal.,prediction
72,NaN,Utilitarianism's current choice for 2018 FIFA #WorldCup championship: Mexico.\n\nWith many caveats.\n\nblog.practicalethics.ox.ac.uk/2018/06/the-ut…,prediction
84,MT Fires,Yay sports illustrated picking ravens to go 7-9 so that means we are going back to the playoffs for sure now hahah\nhttps://www.si.com/nfl/2018/08/21/nfl-preseason-predictions-team-records-playoffs-super-bowl-liii-champion,prediction
96,NaN,Me ⁦@friscojosh⁩ ⁦@Neil_Paine⁩ and ⁦⁦@gwfost⁩ on the Divisional Playoffs and next week’s championship picks. What Star Wars Episode I-III character would Taysom Hill be? https://fivethirtyeight.com/features/what-did-we-learn-from-the-nfl-divisional-playoffs/,prediction


In [30]:
len(bluesky_user_predictions)

45